### Summarizer
The purpose of this code is to interact with a co:here summarizer model via API. 

For my experiment on 4/1/2023, the summarizer will process 20 examples from the EK knowledge base and create summaries for each. 

In [ ]:
# import libraries
import os
from dotenv import load_dotenv 
import pandas as pd
import cohere 
import json
import openai

In [2]:
knowledge_base = '20_EK_articles.json'
with open('20_EK_articles.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data.items(), columns=['url', 'text'])

In [16]:
df.head

<bound method NDFrame.head of                                                   url  \
0   https://enterprise-knowledge.com/5-steps-to-en...   
1   https://enterprise-knowledge.com/applied-knowl...   
2   https://enterprise-knowledge.com/breaking-it-d...   
3   https://enterprise-knowledge.com/data-catalog-...   
4   https://enterprise-knowledge.com/elevating-you...   
5   https://enterprise-knowledge.com/expert-analys...   
6   https://enterprise-knowledge.com/how-does-km-i...   
7   https://enterprise-knowledge.com/is-chatgpt-re...   
8   https://enterprise-knowledge.com/knowledge-cap...   
9   https://enterprise-knowledge.com/knowledge-man...   
10  https://enterprise-knowledge.com/leveraging-he...   
11  https://enterprise-knowledge.com/resolving-uni...   
12  https://enterprise-knowledge.com/the-importanc...   
13  https://enterprise-knowledge.com/the-value-of-...   
14  https://enterprise-knowledge.com/three-pillars...   
15  https://enterprise-knowledge.com/tips-for-impl...   
1

In [3]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")
# openai.Model.list()

In [4]:
# generate completions
responses = pd.Series(index=df.index, dtype='object') # init empty series for responses
for i, j in df.iterrows():
    text = j['text'] # set value from input DataFrame
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Your task is to read the provided article and summarize it into a short, catchy abstract that will attract the attention of executives at large enterprise organizations."},
                {"role": "user", "content": text}
            ],
            max_tokens=250,
            temperature=0.1,
        )
        # print('Summary:', response.summary)
        responses.at[i] = response.choices[0].message['content']
    except openai.InvalidRequestError:
        pass

In [18]:
# save completions to a JSON file
responses.to_json('completions.json', orient='index')